In [ ]:
### 지도시각화 : folium 라이브러리, kakao API를 활용

In [4]:
### 1. 데이터 준비(1_crawling_raw.xlsx)
import pandas as pd

row_total = pd.read_excel('data/1_crawling_raw.xlsx')
row_total.head()

## 위치정보만 가져오기
place_counts = row_total['place'].value_counts()
print(place_counts)

Jeju                    271
Jeju-do                 179
Jungle Book by Alice    108
Seogwipo                 66
제주도 크리스마스 박물관            59
                       ... 
제주마당                      1
형제섬                       1
동복해녀식당                    1
종로 광장시장                   1
디스이즈핫                     1
Name: place, Length: 1028, dtype: int64


In [5]:
### 위치정보를 DataFrame 형태로 변환
place_counts_df = pd.DataFrame(place_counts)
place_counts_df.head()
place_counts_df.info()      # value_counts()로 인해 index가 key값(키워드)으로 들어감

### 위치정보를 excel파일로 저장
place_counts_df.to_excel('data/2_place_counts.xlsx')

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, Jeju to 디스이즈핫
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   place   1028 non-null   int64
dtypes: int64(1)
memory usage: 16.1+ KB


In [6]:
### 지명만 빼오기
places = list(place_counts_df.index)
places

['Jeju',
 'Jeju-do',
 'Jungle Book by Alice',
 'Seogwipo',
 '제주도 크리스마스 박물관',
 '할로비치',
 '제주에인감귤밭',
 'Jeju Island',
 '성산일출봉 城山日出峰  Seongsan Ilchulbong',
 '폼포코식당_pompokokitchen',
 '1100고지',
 'Nimome',
 '월정리해변',
 '고집돌우럭중문점',
 '석부작박물관',
 '제주도 애월읍',
 '알뜨르 비행장',
 '카멜리아 힐',
 '제주레포츠랜드',
 '밥깡패',
 '제주 함덕 서우봉 해변',
 '제주도 서귀포 중문관광단지 濟州島西歸浦中文觀光團地',
 '제주고궁한복카페 jeju gogung hanbok studio',
 '위미동백나무군락지',
 '제주신화월드 Jeju Shinhwa World',
 'Woljeongri Beach',
 '협재해변 Beach',
 '연동 바오젠 거리',
 '김녕미로공원 Jeju Kimnyoung Maze Park',
 '고래배꼽',
 '제주 성산 신산포구자연산횟집 회포장',
 '새별오름',
 '용머리해안',
 '용눈이오름',
 '캔디원',
 '사려니숲길',
 '하이엔드 제주',
 '제주빅볼랜드',
 '서귀포 느영나영 게스트하우스',
 '휴애리 자연생활공원',
 '옹포리',
 '삼무공원',
 '마마뜰',
 '제주어린왕자게스트하우스',
 '위미동백나무군락',
 '제주커피박물관 Baum',
 'Jeju Aewol',
 '한라산',
 '위미2리 동백군락지',
 'Jeju Island 제주특별자치도 济州道',
 '광치기해변',
 '서우봉',
 '제주해남',
 '한라산 (漢拏山, Hallasan)',
 '카페한라산',
 '성이시돌목장',
 '제주 송악산',
 '옹포별장가든',
 '월정리카페콧수염',
 '牛岛 Udo Island 우도',
 '곽지해수욕장',
 '카페브리프',
 '아줄레주',
 '에코랜드',
 '산굼부리',
 '제주 꿈꾸는고래 스쿠버&게스트하우스',
 'Terarosa - 테라로사',

In [7]:
### KAKAO API 사용 (로컬/지도 API) : 장소 검색
import requests

searching = '합정 스타벅스'
url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={searching}'

header = {
    'Authorization': 'KakaoAK 0520670041f5f53769beeb6f3b29e756'
    # host는 ip로 접속하지 않는 이상 반드시 써야할 필요는 없음
}

place = requests.get(url, headers = header).json()['documents'][0]
place

{'address_name': '서울 마포구 서교동 395-166',
 'category_group_code': 'CE7',
 'category_group_name': '카페',
 'category_name': '음식점 > 카페 > 커피전문점 > 스타벅스',
 'distance': '',
 'id': '26572121',
 'phone': '1522-3232',
 'place_name': '스타벅스 서교점',
 'place_url': 'http://place.map.kakao.com/26572121',
 'road_address_name': '서울 마포구 양화로 78',
 'x': '126.916980454434',
 'y': '37.5514601750423'}

In [83]:
### 반복작업 진행 시 진행바 표시하기 위한 라이브러리 tqdm 활용
# pip install tqdm
# pip install ipywidgets
from tqdm.notebook import tqdm
# from tqdm.auto import tqdm -> for문에서 반복문 조건 기준으로 얼마나 진행됐는지 확인


In [8]:
### 카카오 로컬API를 활용한 장소 검색 함수 만들기
## 인자 : searching - 첫 데이터만 가져오기(신뢰성 up)
## name(place_name), x, y
## 반환값 : [name, x, y, searching]

def find_place(searching) :
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={searching}'
    
    header = {'Authorization': 'KakaoAK 0520670041f5f53769beeb6f3b29e756'}    
    
    place = requests.get(url, headers = header).json()['documents'][0]
    name = place['place_name']
    x = place['x']
    y = place['y']
    data = [name, x, y, searching]    
    
    return data

In [ ]:
from tqdm.notebook import tqdm
import ipywidgets

place_info = []

for searching in tqdm(places) :
    try : place_info.append(find_place(searching))
    except : pass
    
print(place_info)

In [13]:
### 생성된 정보를 df로 만들어서 엑셀로 저장

place_info_df = pd.DataFrame(place_info)
place_info_df.columns = ['name_official', '경도', '위도', 'instagram_keyword']
place_info_df.to_excel('data/3_places_info.xlsx', index=False)

In [22]:
### 인스타 게시량 파일(2_place_counts.xlsx), 위치정보(3_places_info.xlsx)

place_counts_df = pd.read_excel('data/2_place_counts.xlsx', index_col=0)    # 인덱스열 지정 - 인덱스에 따라 값이 merge가 됨
place_info_df = pd.read_excel('data/3_places_info.xlsx')

### 병합하기
place_visual = pd.merge(place_info_df, place_counts_df, how='inner', left_on='name_official', right_index = True)
 
place_visual.head()

,name_official,경도,위도,instagram_keyword,place
1,서귀포잠수함,126.558616,33.239303,Seogwipo,1
620,서귀포잠수함,126.558616,33.239303,서귀포잠수함,1
4,제주에인감귤밭,126.539014,33.256561,제주에인감귤밭,48
5,제주도,126.545874,33.379778,Jeju Island,4
84,제주도,126.545874,33.379778,제주도,4


In [24]:
### 데이터 중복 검사
place_visual['name_official'].value_counts()    # 중복데이터 있는지 확인

제주동문재래시장    4
오설록티뮤지엄     3
자매국수 본점     3
서귀포잠수함      2
빌라드아토       2
           ..
쉐프부부        1
이게밥도둑       1
숨비아일랜드      1
삼공주회센터      1
디스이즈핫       1
Name: name_official, Length: 376, dtype: int64

In [35]:
### 장소 이름을 기준으로 병합

place_visual = place_visual.pivot_table(index=['name_official', '경도', '위도'], values='place', aggfunc='sum')
place_visual.head()

,,,place
name_official,경도,위도,
73st,126.455109,33.456989,1
가드망제,126.532954,33.500991,1
가배,126.951868,37.738173,1
가시식당,126.771438,33.353060,2
가시아방국수,126.918065,33.438605,1


In [33]:
### 병합 데이터를 저장(덮어쓰기)
place_visual.to_excel('data/3_places_info.xlsx')

In [37]:
### 정제한 데이터 가져오기

place_data = pd.read_excel('data/3_places_info.xlsx')
place_data.info()
place_data.head()   # >> 다시 가져왔기 때문에 헤더는 인덱스가 아니라 그냥 데이터임 !!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_official  376 non-null    object 
 1   경도             377 non-null    float64
 2   위도             377 non-null    float64
 3   place          377 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 11.9+ KB


,name_official,경도,위도,place
0,73st,126.455109,33.456989,1
1,가드망제,126.532954,33.500991,1
2,가배,126.951868,37.738173,1
3,가시식당,126.771438,33.353060,2
4,가시아방국수,126.918065,33.438605,1


In [62]:
### folium을 이용한 지도 시각화
# pip install folium
import folium

MT_Hanla = [33.362500, 126.533694]
map_jeju = folium.Map(location=MT_Hanla, zoom_start=10)

for i in range(len(place_data)) :
    name = place_data['name_official'][i]   # 공식명칭
    count = place_data['place'][i]          # 게시글 개수
    size = int(count) * 2
    long = float(place_data['위도'][i])
    lat = float(place_data['경도'][i])
    folium.CircleMarker((long, lat), color='purple', radius = size, tooltip = name).add_to(map_jeju)

map_jeju

In [63]:
### 사진 저장
map_jeju.save('data/3_place_info.html')

In [66]:
### 마커(circle) 집합을 통한 지도 표시
from folium.plugins import MarkerCluster

locations = []
names = []

for i in range(len(place_data)) :
    data = place_data.iloc[i]   # 행을 하나씩 처리하기 위함
    locations.append((float(data['위도']), float(data['경도'])))    # 위도, 경도 순으로 정보 저장
    names.append(data['name_official'])
    
MT_Hanla = [33.362500, 126.533694]
map_jeju2 = folium.Map(location=MT_Hanla, zoom_start=11)

maker_cluster = MarkerCluster(locations = locations, popups = names,name='Jeju',
                              overlay=True, control=True)
maker_cluster.add_to(map_jeju2)
folium.LayerControl().add_to(map_jeju2)
map_jeju2

In [67]:
### 만들어진 그룹화 지도 저장
map_jeju2.save('data/3_jeju_cluster.html')

In [69]:
### 특정 단어 포함 게시글 찾기

import pandas as pd
raw_total = pd.read_excel('data/1_crawling_raw.xlsx')
raw_total.head()

# 해돋이 포함된 게시글 찾기

select_word = '해돋이'

check_list = []
for content in raw_total['content'] :
    if select_word in content :
        check_list.append(True)
    else :
        check_list.append(False)

select_df = raw_total[check_list]
select_df.head()

,content,date,like,place,tags
34,..“쿠룽쿠룽 해트맨 마씸!”.제주방언으로 ‘쿨쿨 자던 해가 떴습니다’.다사다난 했...,2019-01-01,89,서귀포 법환포구,[]
73,아듀 2018!🤧👋🏻2019🐷황금돼지해의 첫 일출🌅을 만나볼 수 있는 제주 일출명소...,2018-12-31,18,Jeju-do,"['#제주', '#제주도', '#서귀포', '#제주도민', '#제주여행', '#제주..."
142,4일간 휴 ~ 뭐하지? ㅠ 뭐할까?. #해넘이#해돋이 어디로가지? .누워서보는극장?...,2018-12-28,94,NaN,"['#해넘이', '#해돋이', '#제주', '#jejudo', '#jeju', '#..."
441,순간포착엄청 놀랐네요갑자기 새들이 무더기로 나타나서날아오르길래 ㆍ그쪽에 무슨일 있는...,2018-12-18,77,제주프레임스파빌,"['#제주도공항', '#프레임펜션', '#프레임스파빌', '#제주', '#제주프레임..."
2458,해돋이#제주 #혼저옵서예 #제주스타그램 #제주도 #제주여행 #제주관광 #제주도여행 ...,2018-10-23,81,NaN,"['#제주', '#혼저옵서예', '#제주스타그램', '#제주도', '#제주여행', ..."


In [ ]:
### 선택한 데이터

for i in select_df.index :
    print(select_df.loc[i,'content'])
    print('-'*40)
    

In [75]:
fpath = f'data/4_select_data_{select_word}.xlsx'
select_df.to_excel(fpath, index=False)

In [79]:
### 여러개의 단어 선택/추출/저장
# 선택 단어 리스트 : 해돋이, 박물관, 힐링, 게스트하우스, 섭지코지
# 각 선택 단어를 추출하여 저장 - 4_select_data_{선택단어}.xlsx

def select_keyword(keyword) :        
    raw_total = pd.read_excel('data/1_crawling_raw.xlsx')
    condition = []
    for x in row_total['content'] :
        if keyword in x : condition.append(True)
        else : condition.append(False)
    
    select_df = raw_total[condition]
    fpath = f'data/4_select_data_{keyword}.xlsx'
    select_df.to_excel(fpath, index=False)


In [80]:
keywords = ['해돋이', '박물관', '힐링', '게스트하우스', '섭지코지']

for keyword in keywords :
    select_keyword(keyword)